<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Fake SQL Database class
from typing import Tuple, Sequence, List, Any, Callable, Dict, Iterator
from collections import defaultdict

# A few type aliases we'll use later
Row = Dict[str, Any]
WhereClause = Callable[[Row], bool] # Predicate for a single row
HavingClause = Callable[[List[Row]], bool] # Predicate over multiple ros

In [2]:
# Constructor for class to create column names and types
class Table:
  def __init__(self, columns: List[str], types: List[type]) -> None:
    assert len(columns) == len(types), "# of columns must == # of types"
    self.columns = columns
    self.types = types
    self.rows: List[Row] = [] # no data yet

  # Helper method to get the type of a column
  def col2type(self, col: str) -> type:
    idx = self.columns.index(col) # Find the index of the column
    return self.types[idx]
    